There are various ways to use the pre-trained model...

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import gzip
import os
import sys
import time

import numpy
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data

IMAGE_SIZE = 28
NUM_CHANNELS = 1
PIXEL_DEPTH = 255
NUM_LABELS = 10
def data_type(): return tf.float32

Load Data:

In [4]:
data_dir = 'data'
mnist = input_data.read_data_sets(data_dir, one_hot=True)

test_data = mnist.test.images
test_labels = mnist.test.labels

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


In [5]:
"""
Q1: should I reshape my data? (1, 784) or (1, 28, 28) or (None, 28, 28, 1) or (64, 28, 28, 1)? 
(None, ) or (None 10) for the layer?
-- that requires a way to know the spec of input layer

Now assume I know that the input layer is (None, 28, 28, 1) for image and (None, 10) for the labels, so...
"""

test_data = test_data.reshape(test_data.shape[0], 28, 28, 1)

In [ ]:
"""
Q2: Let's assume that there is a convenient "accuracy" function in the original 
-- and we know that, with all its description, then it's good. Let's not re-invent the wheel.
So that requires a method to add these variables during model-saving and a method to get them out.
"""

with tf.Session() as sess:
    
    # Load graph, including the definition of "accuracy".
    # Load weight
    
    print('Validation error: %.1f%%' % accuracy.eval(feed_dict={
                train_data_node: test_data,
                train_labels_node: test_labels
    }))

In [ ]:
"""
But that may not happen -- the trainer may not define the "accuracy" parameter at all, 
or that the shape of "accuracy" tensor is not suitable here, 
which are both quite reasonable. So the inferencer have to build from loaded models.
We assume the "logits" part is always necessary in a saved model, so...
"""

# We need to define suitable labels, with known parameter
test_labels_node = tf.placeholder(
      tf.float32, #  data_type() 
      shape=(None, 28, 28, 1))

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
      labels=test_labels_node, logits=logits))

# ALL of these weights' name(fc_*)and "where" they are 
# -- which means the model should be visualized for the inferencer to use

regularizers = (tf.nn.l2_loss(fc1_weights) + tf.nn.l2_loss(fc1_biases) +
                tf.nn.l2_loss(fc2_weights) + tf.nn.l2_loss(fc2_biases))

loss += 5e-4 * regularizers
batch = tf.Variable(0, dtype=tf.float32)  # data_type()) -- again, need to know the precision

learning_rate = tf.train.exponential_decay(
      0.01,                # Base learning rate.
      batch * BATCH_SIZE,  # Current index into the dataset.
      test_labels.shape[0],          # Decay step.
      0.95,                # Decay rate.
      staircase=True)
optimizer = tf.train.MomentumOptimizer(learning_rate, 0.9).minimize(loss, global_step=batch)
prediction = tf.nn.softmax(logits)
correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(test_labels_node, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# And then the normal use 

with tf.Session() as sess:
    
    # Load graph, including the definition of "accuracy".
    # Load weight
    
    print('Validation error: %.1f%%' % accuracy.eval(feed_dict={
        # Note that even though we know the test_lables_name now, 
        # the train_data_node as input layer is still hidden behind the
        # definition of "logit".
        train_data_node: test_data, 
        test_labels_node: test_labels        
    }))

In [ ]:
"""
But in cases I may find myself not happy with the input format, and need to change it to my own input shape.
And in this example, we may also want to use another way to 
"""

# Load graph.

EVAL_BATCH_SIZE = 64 # Should we also load some constant? Maybe
  
# This step requires the change of the first input layer to eval_data:
# train_data_node = tf.placeholder(
#      data_type(),
#      shape=(BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS))
#  eval_data = tf.placeholder(
#      data_type(),
#      shape=(EVAL_BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS))
# They differ in input batch size

eval_prediction = tf.nn.softmax(model(eval_data))

# This function is just an example of a different method the inferencer want to use 
# to measure accuracy.
def error_rate(predictions, labels):
    return 100.0 - (
      100.0 *
      numpy.sum(numpy.argmax(predictions, 1) == labels) /
      predictions.shape[0])

def eval_in_batches(data, sess):
    size = data.shape[0]
    if size < EVAL_BATCH_SIZE:
          raise ValueError("batch size for evals larger than dataset: %d" % size)
    predictions = numpy.ndarray(shape=(size, NUM_LABELS), dtype=numpy.float32)
    for begin in xrange(0, size, EVAL_BATCH_SIZE):
        end = begin + EVAL_BATCH_SIZE
        if end <= size:
            predictions[begin:end, :] = sess.run(
                eval_prediction,
                feed_dict={eval_data: data[begin:end, ...]})
        else:
            batch_predictions = sess.run(
                eval_prediction,
                feed_dict={eval_data: data[-EVAL_BATCH_SIZE:, ...]})
            predictions[begin:, :] = batch_predictions[begin - size:, :]
    return predictions

with tf.Session() as sess:
    # Load weight
    test_error = error_rate(eval_in_batches(test_data, sess), test_labels)
    print('Test error: %.1f%%' % test_error)

In [ ]:
"""
Also in many cases, we may want to drop some layers 
-- such as the dropout layer in a training model but unnecessary in a test model;
we may also want to drop/change the output layer (and add more) layers.

We call it "Layer Manipulation". That must be supported. 
"""

In [ ]:
"""
Re-train/Fine-tune a model, there might be two cases:
1) I want to fine-tune the existing MNIST model for, say, notMNIST dataset,
which means Inference might not be the only purpose here; 
2) I want to use InceptionV3 for MNIST -- leave out of the scope now... 
"""